# Discover and access client app workspaces

This notebook is intended for development and testing purpose only. It shall allow you to discover the available workspaces and access them on behalf of a client application.

The use case is an application that needs to access TerrAPI in order to upload data to a workspace from a client application.

In this notebook, we will:
- Authenticate with the API as the client app in order to get an access token
- Get the user information from TerrAPI
- List the workspaces of the user
- Select a workspace and access it
- Upload a file to the workspace
- Get a download link for the file (signed url)

In [22]:
#################################################
# First Let's set up the environment
iam_url = 'https://iam.terradue.com/realms/master'
terrapi_url = 'https://api.terradue.com/core'
platform_scopes = ['gep']  # list of platform scopes. See https://api.bios-dev.terradue.com/core/docs/platforms/ for more information

# Set localdev to True if you are running the notebook to services running on your local machine (self-signed certificates)
localdev = False
#################################################

import sys, os
import pprint
sys.path.append('../')
import openapi_client
from openapi_client.rest import ApiException

configuration = openapi_client.Configuration(
    host = terrapi_url
)
configuration.verify_ssl = not(localdev)


## Authentication

The following code will authenticate with the TerrAPI and get an access token. You can find more information in the [TerrAPI documentation](https://docs.terrapia.io/docs/authorization).

In [23]:
from IPython.display import JSON
from authlib.jose import jwt
import requests


pp = pprint.PrettyPrinter(indent=4, width=80, compact=True)

# First request an access token from keycloak using the client credentials flow
# and use the access token to create an authenticated client
# the client will automatically refresh the token when it expires

from authlib.oidc.discovery import well_known, OpenIDProviderMetadata
t2_oidc_dicovery_url = well_known.get_well_known_url(iam_url, external=True)
t2_oidc_dicovery = OpenIDProviderMetadata(requests.get(t2_oidc_dicovery_url, verify=not(localdev)).json())

# We will use the public client of Terradue IAM
client_id = 'app_client'
client_secret = 'app_client_secret'
# Scopes requested (the platform related scopes must be set accordingly)
scopes = 'openid email profile offline_access' + ' '.join([' {}'.format(s) for s in platform_scopes])

# using requests implementation
from authlib.integrations.requests_client import OAuth2Session
from authlib.common.security import generate_token
client = OAuth2Session(client_id, client_secret, scope=scopes)
redirect_uri='http://localhost:8888/terrapi/'

authorization_endpoint = t2_oidc_dicovery['authorization_endpoint']
token_endpoint = t2_oidc_dicovery['token_endpoint']
token = client.fetch_token(token_endpoint, grant_type='client_credentials')
os.environ['ACCESS_TOKEN'] = token['access_token']
os.environ['REFRESH_TOKEN'] = token['refresh_token']
pp.pprint(token)

{   'access_token': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJkbmp4eGIyZk0ybE9Tc1RfYS02enktMmNHRVo3SzN1aWYwX2pueGk3akk0In0.eyJleHAiOjE3MTg2MTM5OTEsImlhdCI6MTcxODYxMDM5MSwianRpIjoiZDQ2MGNmMTItYmU1Zi00MzljLWJkMWEtODAyOGQyM2FiNmYyIiwiaXNzIjoiaHR0cHM6Ly9pYW0udGVycmFkdWUuY29tL3JlYWxtcy9tYXN0ZXIiLCJhdWQiOlsiZ2VwLWJpb3MtbWluaW8iLCJlbGxpcC1iaW9zLWFwaSIsImFjY291bnQiXSwic3ViIjoiNWUxMmQwMTktYjNkYi00YWQ4LTk3OTctYTA5NmNjZjFiZWQ0IiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiZ2VwLWVvc3QtYTJzLWhwYy1hcHAiLCJzZXNzaW9uX3N0YXRlIjoiNGRjNWQwNzItMGYxOC00MDQ3LThkMTEtZjM0Y2ExNWNjZmE0IiwiYWNyIjoiMSIsImFsbG93ZWQtb3JpZ2lucyI6WyIvKiJdLCJyZXNvdXJjZV9hY2Nlc3MiOnsiZ2VwLWJpb3MtbWluaW8iOnsicm9sZXMiOlsiczMtdXNlciJdfSwiZWxsaXAtYmlvcy1hcGkiOnsicm9sZXMiOlsiZ2VwLXVzZXIiXX0sImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoib2ZmbGluZV9hY2Nlc3MgZ2VwIGVtYWlsIG9wZW5pZCBwcm9maWxlIiwic2lkIjoiNGRjNWQwNzItMGYxOC00MDQ3LThkMTEtZjM0Y2ExNWNjZmE0IiwiZW1haWxfdmVyaWZpZWQiOmZhbHN

Now you usually have an **ACCESS TOKEN** that we will use to make requests to the API. The access token is actually a JWT token that contains the user's information. The token is signed by the server and can be verified by the server. The token is also encrypted so that the user's information is not visible to anyone else.
Let's use it to get the user's information.

In [24]:
configuration.access_token = os.environ["ACCESS_TOKEN"]
# Enter a context with an instance of the API client
with openapi_client.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = openapi_client.UserApi(api_client)
    job_id = 'job_id_example' # str | 

    try:
        # Get the user information
        userinfo = api_instance.v2_user_info_get()
        print("The response of UserAPI:\n")
        pp.pprint(userinfo)
    except ApiException as e:
        print("Exception when calling UserAPI: %s\n" % e)


The response of UserAPI:

PrincipalContext(tokens=AccessToken(id='346d7650-a386-49d4-828e-5577d6f8a2ee', principal_id='5e12d019-b3db-4ad8-9797-a096ccf1bed4', token='eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJkbmp4eGIyZk0ybE9Tc1RfYS02enktMmNHRVo3SzN1aWYwX2pueGk3akk0In0.eyJleHAiOjE3MTg2MTQwODksImlhdCI6MTcxODYxMDQ4OSwianRpIjoiMzQ2ZDc2NTAtYTM4Ni00OWQ0LTgyOGUtNTU3N2Q2ZjhhMmVlIiwiaXNzIjoiaHR0cHM6Ly9pYW0udGVycmFkdWUuY29tL3JlYWxtcy9tYXN0ZXIiLCJhdWQiOlsiZ2VwLWJpb3MtbWluaW8iLCJhY2NvdW50Il0sInN1YiI6IjVlMTJkMDE5LWIzZGItNGFkOC05Nzk3LWEwOTZjY2YxYmVkNCIsInR5cCI6IkJlYXJlciIsImF6cCI6ImVsbGlwLWJpb3MtYXBpIiwic2Vzc2lvbl9zdGF0ZSI6ImRjZWM3NzZmLTU0MTktNDFiZC1hYTVhLTE0NjdmODEzMTNiNCIsImFsbG93ZWQtb3JpZ2lucyI6WyJodHRwczovL2FwaS50ZXJyYWR1ZS5jb20iXSwicmVzb3VyY2VfYWNjZXNzIjp7ImVsbGlwLWJpb3MtYXBpIjp7InJvbGVzIjpbImdlcC11c2VyIl19fSwic2NvcGUiOiJvcGVuaWQgb2ZmbGluZV9hY2Nlc3MgZ2VwIGVsbGlwIGVtYWlsIHByb2ZpbGUiLCJzaWQiOiJkY2VjNzc2Zi01NDE5LTQxYmQtYWE1YS0xNDY3ZjgxMzEzYjQiLCJlbWFpbF92ZXJpZmllZCI6ZmFsc2UsInByZWZlcnJlZF9

## List workspaces

We will use the access token to get the available workspaces. We will then select a workspace and access it.

In [32]:
configuration.access_token = os.environ["ACCESS_TOKEN"]
# Enter a context with an instance of the API client
with openapi_client.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = openapi_client.StorageApi(api_client)

    try:
        # Get the storage information
        workspaces = api_instance.get_workspaces()
    except ApiException as e:
        print("Exception when calling StorageApi: %s\n" % e)


# We select the service workspace of the current platform
# Usually the default workspace is the one with the naming convention <platform>-<username>-private-workspace
workspace = next(filter(lambda x:x.name == 'gep-processing-results-a2s-service-workspace',workspaces))

pp.pprint(workspace)

workspce_info = api_instance.get_workspace_by_id(workspace.name)
                                           
pp.pprint(workspce_info)

SelfInfo(type=<AuthResourceType.URN_COLON_RESOURCE_COLON_STORAGE_COLON_WORKSPACE: 'urn:resource:storage:workspace'>, name='gep-processing-results-a2s-service-workspace', var_self='https://api.terradue.com/core/v2/storage/workspaces/gep-processing-results-a2s-service-workspace')
IWorkspace(storage_type=<StorageType.S3: 's3'>, storage_point_uri=None, service_uri=None, initialized=None, end_point='https://s3.gep.terradue.com/', remote_id=None, resource_server=None, owner='ellip-bios-api', type=<AuthResourceType.URN_COLON_RESOURCE_COLON_STORAGE_COLON_WORKSPACE: 'urn:resource:storage:workspace'>, status=IResourceStatus(status_code=<ResourceStatusCode.OK: 'OK'>, message='From Authorization Resource'), resource_uris=['s3://gep-processing-results/a2s/'], scopes=['s3:GetObject', 's3:PutObject', 's3:ListBucket', 's3:DeleteObject', 's3:GetBucketLocation'], properties={'bucket': ['gep-processing-results'], 'storage_type': ['s3'], 'service_url': ['https://s3.gep.terradue.com/'], 'platform_id': ['ge

## Get a Security Token to access the workspace

In order to access the workspace via the storage provider, we need to get a security token. The security token is a set of credentials generated signed by the API and that can be verified by the storage provider.

In [35]:
configuration.access_token = os.environ["ACCESS_TOKEN"]
# Enter a context with an instance of the API client
with openapi_client.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = openapi_client.StorageApi(api_client)

    try:
        # Request a security token to access the workspace
        token = api_instance.get_storage_sts(workspace.name)
    except ApiException as e:
        print("Exception when calling StorageApi: %s\n" % e)
        
pp.pprint(token)

StsStorage(storage=IStoragePoint(storage_type=<StorageType.S3: 's3'>, storage_point_uri=None, service_uri=None, initialized=None, end_point='https://s3.gep.terradue.com/', remote_id=None, resource_server=None, owner='ellip-bios-api', type=<AuthResourceType.URN_COLON_RESOURCE_COLON_STORAGE_COLON_WORKSPACE: 'urn:resource:storage:workspace'>, status=IResourceStatus(status_code=<ResourceStatusCode.OK: 'OK'>, message='From Authorization Resource'), resource_uris=['s3://gep-processing-results/a2s/'], scopes=['s3:GetObject', 's3:PutObject', 's3:ListBucket', 's3:DeleteObject', 's3:GetBucketLocation'], properties={'bucket': ['gep-processing-results'], 'storage_type': ['s3'], 'service_url': ['https://s3.gep.terradue.com/'], 'platform_id': ['gep'], 'storage_url': ['s3://gep-processing-results/a2s/'], 'key': ['a2s/'], 'resource_server_id': ['ellip-bios-api']}, platform_id=None, name='gep-processing-results-a2s-service-workspace', var_self=None), credentials=IStorageSTS(access_key='IBASGDXIT3WV3VBA

## Access the workspace using the token

We will use the security token to access the workspace and upload a file. All the information about the storage access and credentials are contained in the security token.
In this case, it is a S3 storage provider, so we will use the `boto3` library to access the workspace using the parameters in the security token.

In [34]:
import boto3
import boto3.session

## Create a S3 session with the service URL and the security token
session = boto3.session.Session(aws_access_key_id=token.credentials.access_key, 
                                aws_secret_access_key=token.credentials.secret_key, 
                                aws_session_token=token.credentials.token,
                                region_name=token.storage.properties.__contains__('region') and token.storage.properties['region'][0] or 'us-east-1')

## Create a S3 client with the service URL and the security token
s3 = session.client('s3', endpoint_url=token.storage.properties['service_url'][0], verify=not(localdev))

pp.pprint(token.storage.service_uri)
pp.pprint(token.storage.properties)
pp.pprint(token.credentials.token)

## List the content of the workspace
response = s3.list_objects_v2(Bucket=token.storage.properties['bucket'][0], Prefix=token.storage.properties['key'][0])
pp.pprint(response)

None
{   'bucket': ['gep-processing-results'],
    'key': ['a2s/'],
    'platform_id': ['gep'],
    'resource_server_id': ['ellip-bios-api'],
    'service_url': ['https://s3.gep.terradue.com/'],
    'storage_type': ['s3'],
    'storage_url': ['s3://gep-processing-results/a2s/']}
'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiIyQ1E4NFo5QUVBSTRVNVoxRFRYWCIsImFsbG93ZWQtb3JpZ2lucyI6WyJodHRwczovL2FwaS50ZXJyYWR1ZS5jb20iXSwiYXVkIjpbImdlcC1iaW9zLW1pbmlvIiwiYWNjb3VudCJdLCJhenAiOiJlbGxpcC1iaW9zLWFwaSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwiZXhwIjoxNzE4NjEzMTUxLCJpYXQiOjE3MTg2MTIyNTEsImlzcyI6Imh0dHBzOi8vaWFtLnRlcnJhZHVlLmNvbS9yZWFsbXMvbWFzdGVyIiwianRpIjoiMGYwODY1NjgtOTAzYi00YjNhLTljZmUtNDUyMzE0ZjIxYTQ5IiwibWluaW9fcG9saWN5IjoiZ2VwLXByb2Nlc3NpbmctcmVzdWx0cy1hMnMtc2VydmljZS13b3Jrc3BhY2UtbWFuYWdlci51c2VyLmZyb20taWFtLHNoYXJlZC13aXRoLXBsYXRmb3JtLWdlcC5wbGF0Zm9ybS5mcm9tLWlhbSIsInBsYXRmb3JtcyI6eyJlbGxpcCI6eyJpZCI6ImVsbGlwIn0sImdlcCI6eyJpZCI6ImdlcCJ9fSwicHJlZmVycmVkX3VzZXJuYW1lIjoic2VydmljZS1hY2NvdW50LW